In [1]:
import os
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import numpy as np
import codecs
import pdb
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

In [2]:
directory=os.getcwd()
webpage_dump = directory + "\\webpages"

In [ ]:
driver=webdriver.Chrome()
url="https://www.amazon.in/s?k=monitors&crid=IBS3D4BWHX6M&qid=1675433952&sprefix=monitors%2Caps%2C240&ref=sr_pg_1"
driver.get(url)
soup=BeautifulSoup(driver.page_source, "html.parser")
page_number=soup.find_all(class_="s-pagination-strip")[0].text
Nfinder=page_number.find("N")
Page=page_number[Nfinder-2] + page_number[Nfinder-1]
Page=Page.strip(".")
Page_count=int(Page)
for i in range(1,Page_count+1):
    url=f"https://www.amazon.in/s?k=monitors&page={i}&crid=IBS3D4BWHX6M&qid=1675433952&sprefix=monitors%2Caps%2C240&ref=sr_pg_{i}"
    driver.get(url)
    a=os.path.join(webpage_dump, f"{i}.html")
    b=codecs.open(a, "w", "utf-8")
    c=driver.page_source
    b.write(c)

In [ ]:
Original_Price=[]
Renewed=[]
Discount=[]
Discount_Percentage=[]
Discounted_Price=[]
Company_Name=[]
Average_Rating=[]
Ratings_Count=[]
Model_Number=[]
Delivery_Date=[]
Screen_Size=[]
Resolution=[]
resolution_numbers=["1080", "1440", "2160", "1366", "1280", "1600"]
resolution_text=["FHD", "QHD", "UHD", "5K", "HD"]
resolution=["FHD", "2K", "4K", "5K"]
Additional_Details=[]
with open(f"{webpage_dump}\\{1}.html", "r", encoding="UTF-8") as reader:
    html_string=reader.read()
    soup=BeautifulSoup(html_string, 'html.parser')
page_number=soup.find_all(class_="s-pagination-strip")[0].text
Nfinder=page_number.find("N")
Page=page_number[Nfinder-2] + page_number[Nfinder-1]
Page=Page.strip(".")
Page_count=int(Page)
for z in range(1,Page_count+1):
    print(f"Processing Page Number: {z} \n")
    with open(f"{webpage_dump}\\{z}.html", "r", encoding="UTF-8") as reader:
        html_string=reader.read()
        soup=BeautifulSoup(html_string, 'html.parser')
    d=soup.find_all(class_="a-section a-spacing-small a-spacing-top-small")[0].text
    d=d.strip("\n")
    e=d.split("-")
    master_result=e[1].split(" ")
    f=int(master_result[0])-int(e[0])
    w=f
    count=0
    for i in range(w):
        f=soup.find_all(class_="sg-col sg-col-4-of-12 sg-col-8-of-16 sg-col-12-of-20 s-list-col-right")[i].text
        if "Sponsored" in f:
            count+=1
    w=w+count
    w=w+1
    for i in range(w):
        d=soup.find_all(class_="a-size-medium a-color-base a-text-normal")[i].text
        d=d.split(" ")
        if (d[0]=="Renewed"):
            Company_Name.append(d[1])
        else:
            Company_Name.append(d[0])
    for i in range(w):
        f=soup.find_all(class_="sg-col sg-col-4-of-12 sg-col-8-of-16 sg-col-12-of-20 s-list-col-right")[i].text
        f=f.strip("SponsoredSponsored You are seeing this ad based on the product’s relevance to your search query.Let us know")
        if resolution[0] in f:
            Resolution.append(resolution_text[0])
        elif resolution[1] in f:
            Resolution.append(resolution_text[1])
        elif resolution[2] in f:
            Resolution.append(resolution_text[2])
        elif resolution_numbers[0] in f:
            Resolution.append(resolution_text[0])
        elif resolution_numbers[1] in f:
            Resolution.append(resolution_text[1])
        elif resolution_numbers[2] in f:
            Resolution.append(resolution_text[2])
        elif resolution_text[0] in f:
            Resolution.append(resolution_text[0])
        elif resolution_text[1] in f:
            Resolution.append(resolution_text[1])
        elif resolution_text[2] in f:
            Resolution.append(resolution_text[2])
        elif resolution_text[3] in f:
            Resolution.append(resolution_text[3])
        elif resolution_numbers[3] in f:
            Resolution.append(resolution_text[4])
        elif resolution_numbers[4] in f:
            Resolution.append(resolution_text[4])
        elif resolution_numbers[5] in f:
            Resolution.append(resolution_text[4])
        else:
            Resolution.append(np.nan)
    x=0
    y=0
    while(x!=w):
        f=soup.find_all(class_="sg-col sg-col-4-of-12 sg-col-8-of-16 sg-col-12-of-20 s-list-col-right")[x].text
        f=f.strip("SponsoredSponsored You are seeing this ad based on the product’s relevance to your search query.Let us know")
        if ("₹") in f:
            d=soup.find_all(class_="a-row a-size-base a-color-base")[y].text
            d=d.split("₹")
            
            g=d[1].replace(",", "")
            if (len(d)<4):
                Discount_Percentage.append(np.nan)
                Discount.append(np.nan)
                Discounted_Price.append(np.nan)
                Original_Price.append(float(g))
                h=float(e)
            else:
                e=d[3].replace(",", "")
                Discounted_Price.append(float(g))
                h=float(e)
                k=float(g)
                Original_Price.append(float(e)) 
                
                
                Discount.append(h-k)
                Discount_Percentage.append(int(((h-k)/h)*100))
            y+=1       
        else:
            Original_Price.append(np.nan)
            Discount.append(np.nan)
            Discount_Percentage.append(np.nan)
            Discounted_Price.append(np.nan)
        x+=1 
    x=0
    y=0
    while(x!=w):
        f=soup.find_all(class_="sg-col sg-col-4-of-12 sg-col-8-of-16 sg-col-12-of-20 s-list-col-right")[x].text
        f=f.strip("SponsoredSponsored You are seeing this ad based on the product’s relevance to your search query.Let us know")
        try:
            if "out of 5 stars" in f:
                d=soup.find_all(class_="a-row a-size-small")[y].text
                d=d.split(" ")
                Average_Rating.append(float(d[0]))
                if (len(d[5])>3):
                    e=d[5].split(",")
                    Ratings_Count.append(int(e[0]+e[1]))
                else:
                    Ratings_Count.append(int(d[5]))
                y+=1
            else:
                Average_Rating.append(np.nan)
                Ratings_Count.append(np.nan)
            x+=1
        except IndexError:
            continue
    for i in range(w):
        f=soup.find_all(class_="sg-col sg-col-4-of-12 sg-col-8-of-16 sg-col-12-of-20 s-list-col-right")[i].text
        f=f.strip("SponsoredSponsored You are seeing this ad based on the product’s relevance to your search query.Let us know")
        e=f.split(" ")
        Model_Number.append(e[1])
    x=0
    y=0
    months=["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
    month_number=["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
    while (x!=w):
        f=soup.find_all(class_="sg-col sg-col-4-of-12 sg-col-8-of-16 sg-col-12-of-20 s-list-col-right")[x].text
        if 'Get' in f:
            d=soup.find_all(class_="a-color-base a-text-bold")[y].text
            d=d.split(",")
            e=d[1].split(" ")
            g=months.index(e[1])
            
            if len(e[2])==1:
                h="0"+e[2]
                k=f"2023-{month_number[g]}-{h}"  
            else:
                h=e[2]
                k=f"2023-{month_number[g]}-{h}"
            Delivery_Date.append(k)
            y+=1
        
        else:
            Delivery_Date.append(np.nan)
        x+=1
    for i in range(w):
        f=soup.find_all(class_="a-size-mini a-spacing-none a-color-base s-line-clamp-2")[i].text
        Additional_Details.append(f)
    for i in range(w):
        f=soup.find_all(class_="sg-col sg-col-4-of-12 sg-col-8-of-16 sg-col-12-of-20 s-list-col-right")[i].text
        if "(Renewed)" in f:
            Renewed.append("Yes")
        else:
            Renewed.append(np.nan)
    print(f"Page Number {z} Complete.")
df=pd.DataFrame({"Company_Name": Company_Name, "Model_Number": Model_Number, "Resolution": Resolution, "Original_Price": Original_Price, "Discount": Discount, "Discount_Percentage": Discount_Percentage, "Discounted_Price": Discounted_Price, "Average_Rating": Average_Rating, "Ratings_Count": Ratings_Count, "Delivery_Date": Delivery_Date, "Renewed_Staus": Renewed, "Additional_Details": Additional_Details})
df.to_excel("Amazon Monitor Scrap.xlsx", sheet_name="1")    
print("Success!")

In [ ]:
df=pd.read_excel("Amazon Monitor Scrap.xlsx", sheet_name="1")
df

In [ ]:
Company_Names=df.Company_Name.unique().tolist()
Company_Names

In [ ]:
df[df["Company_Name"]=="Parikshit"]

In [ ]:
new_df=df.drop(df["Company_Name"][df["Company_Name"]=="ASUS"].count()<5, axis=0)

In [ ]:
df=df.drop(df[df["Original_Price"]<4000].index, axis=0)

In [ ]:
df=df.drop(df[df["Company_Name"]=="X"].index, axis=0)

In [ ]:
df=df.drop(df[df["Company_Name"]=="ELS-VS16HD"].index, axis=0)

In [ ]:
df=df.drop(df[df["Company_Name"]=="ZEBRONICS\xa060.4cms\xa024inch"].index, axis=0)

In [ ]:
df=df.drop(df[df["Company_Name"]=="60.4cms"].index, axis=0)

In [ ]:
df=df.drop(df[df["Company_Name"]=="ZEBRONICS\xa068.5cms"].index, axis=0)

In [ ]:
df=df.drop(df[df["Company_Name"]=="22''"].index, axis=0)

In [ ]:
df=df.drop(df[df["Company_Name"]=="(Renewed)"].index, axis=0)

In [ ]:
df

In [ ]:
df.drop(columns=["Unnamed: 0", "Renewed_Staus"], inplace=True)

In [ ]:
df=df.drop(df[df["Original_Price"]=="NaN"].index, axis=1)

In [ ]:
df=pd.read_excel("Amazon Monitor Scrap cleaned.xlsx", sheet_name="2")
df

In [ ]:
a=df.Company_Name.unique().tolist()
g=df.groupby("Company_Name")
c=[]
for i in range(0, len(a)):
    b=g.get_group(a[i]).Company_Name.count()
    c.append(b)

In [ ]:
len(a)

In [ ]:
d=len(c)
others=0
for i in range(0,len(c)):
    print(i)
    try:
        if c[i]<20:
            others+=c[i]
            c.pop(i)
            a.pop(i)
    except IndexError:
        break
c.append(others)
a.append("Others")

In [ ]:
others=0
for i in c:
    if i<6:
        others+=i
        c.remove(i)       


In [ ]:
c

In [ ]:
plt.pie(c, labels=a, autopct='%1.1f%%')
plt.title('Company_share')
plt.show()

In [ ]:
f = plt.subplots()
plt.pie(c, labels=a, radius=3, center=(4, 4),
       wedgeprops={"linewidth": 1, "edgecolor": "white"}, frame=True)
plt.legend()

In [22]:
df2=pd.read_excel("Amazon Monitor Scrap cleaned.xlsx", sheet_name="2")
df2

,Unnamed: 0,Company_Name,Model_Number,Resolution,Original_Price,Discount,Discount_Percentage,Discounted_Price,Average_Rating,Ratings_Count,Delivery_Date,Additional_Details
0,0,BenQ,GW2480,FHD,13990.0,4240.0,30.0,9750.0,4.4,18476.0,2023-02-05,"BenQ GW2480 24-Inch (60.5 Cm) 1080p, LED 1920 ..."
1,1,BenQ,Gw2280,FHD,12990.0,5300.0,40.0,7690.0,4.3,439.0,2023-02-09,BenQ Gw2280 22 Inch (55.88 Cm) 1920 x 1080 Pix...
2,2,Acer,EK220Q,FHD,13750.0,7501.0,54.0,6249.0,4.2,2242.0,2023-02-05,Acer EK220Q 21.5 Inch (54.61 Cm) Full HD (1920...
3,3,Samsung,24-inch(60.46cm),FHD,19110.0,9711.0,50.0,9399.0,4.3,2709.0,2023-02-05,"Samsung 24-inch(60.46cm) FHD Monitor, IPS, 75 ..."
4,4,ZEBRONICS,24,FHD,24999.0,16500.0,66.0,8499.0,3.7,233.0,2023-02-05,ZEBRONICS 24 Inch (60.4 Cm) Widescreen 1920 X ...
...,...,...,...,...,...,...,...,...,...,...,...,...
302,350,ViewSonic,BRATECK,NaN,33600.0,10101.0,30.0,23499.0,4.2,2655.0,2023-02-08,LUMI BRATECK Single Monitor LCD Stand for 13-3...
303,352,BenQ,LED,HD,49990.0,15000.0,30.0,34990.0,4.6,551.0,2023-02-05,Acer LED Backlit Computer LCD Monitor with VGA...
304,353,Acer,Zebronics,NaN,11600.0,4801.0,41.0,6799.0,4.0,234.0,2023-02-05,(Renewed) Zebronics 19.5 inch Monitor(49.5CM) ...
305,355,BenQ,Q24H-10,QHD,119990.0,50000.0,41.0,69990.0,4.4,87.0,2023-02-06,Lenovo Q-Series Q24H-10 24 Inch(60.45Cm) LED 2...


In [ ]:
a=df2.Company_Name.unique().tolist()
g=df2.groupby("Company_Name")
for i in range(len(a)):
    b=1
    c=g.get_group(a[i]).Company_Name
    d=range(1,len(c)+1)
    for j in range(0,len(c)):
        c["Company_sum"]=d[j]

In [ ]:
df2.to_excel("Amazon Monitor Scrap better cleaned.xlsx", sheet_name="1")

In [ ]:
g.get_group("BenQ")

In [43]:
average_price=[]
a=df2.Company_Name.unique().tolist()
g=df2.groupby("Company_Name")
for i in range(len(a)):
    c=g.get_group(a[i]).Original_Price.mean()
    average_price.append(c)

In [55]:
df3=df2.dropna(axis=0)

In [56]:
df3.drop(columns="Unnamed: 0", inplace=True)

C:\Users\aakan\AppData\Local\Temp\ipykernel_12552\247350144.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.drop(columns="Unnamed: 0", inplace=True)


In [58]:
df3.to_excel("Amazon Monitor Scrap master cleaned.xlsx", sheet_name="1", index=False)

In [57]:
df3

,Company_Name,Model_Number,Resolution,Original_Price,Discount,Discount_Percentage,Discounted_Price,Average_Rating,Ratings_Count,Delivery_Date,Additional_Details
0,BenQ,GW2480,FHD,13990.0,4240.0,30.0,9750.0,4.4,18476.0,2023-02-05,"BenQ GW2480 24-Inch (60.5 Cm) 1080p, LED 1920 ..."
1,BenQ,Gw2280,FHD,12990.0,5300.0,40.0,7690.0,4.3,439.0,2023-02-09,BenQ Gw2280 22 Inch (55.88 Cm) 1920 x 1080 Pix...
2,Acer,EK220Q,FHD,13750.0,7501.0,54.0,6249.0,4.2,2242.0,2023-02-05,Acer EK220Q 21.5 Inch (54.61 Cm) Full HD (1920...
3,Samsung,24-inch(60.46cm),FHD,19110.0,9711.0,50.0,9399.0,4.3,2709.0,2023-02-05,"Samsung 24-inch(60.46cm) FHD Monitor, IPS, 75 ..."
4,ZEBRONICS,24,FHD,24999.0,16500.0,66.0,8499.0,3.7,233.0,2023-02-05,ZEBRONICS 24 Inch (60.4 Cm) Widescreen 1920 X ...
...,...,...,...,...,...,...,...,...,...,...,...
300,Dell,VG2455,FHD,18999.0,8259.0,43.0,10740.0,4.7,109.0,2023-02-09,ViewSonic VG2455 24 Inch IPS 1080p Monitor wit...
301,ViewSonic,Crossover,QHD,28800.0,11801.0,40.0,16999.0,4.5,202.0,2023-02-08,ViewSonic Crossover Monitor Vx3276-2K-Mhd-2 32...
303,BenQ,LED,HD,49990.0,15000.0,30.0,34990.0,4.6,551.0,2023-02-05,Acer LED Backlit Computer LCD Monitor with VGA...
305,BenQ,Q24H-10,QHD,119990.0,50000.0,41.0,69990.0,4.4,87.0,2023-02-06,Lenovo Q-Series Q24H-10 24 Inch(60.45Cm) LED 2...


In [61]:
a=df3.Company_Name.unique().tolist()
g=df3.groupby("Company_Name")
for i in range(len(a)):
    b=1
    c=g.get_group(a[i]).Company_Name
    d=range(1,len(c)+1)
    for j in range(0,len(c)):
        df3["Company_sum"]=d[j]

C:\Users\aakan\AppData\Local\Temp\ipykernel_12552\2502708053.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3["Company_sum"]=d[j]


In [62]:
df3

,Company_Name,Model_Number,Resolution,Original_Price,Discount,Discount_Percentage,Discounted_Price,Average_Rating,Ratings_Count,Delivery_Date,Additional_Details,Company_sum
0,BenQ,GW2480,FHD,13990.0,4240.0,30.0,9750.0,4.4,18476.0,2023-02-05,"BenQ GW2480 24-Inch (60.5 Cm) 1080p, LED 1920 ...",1
1,BenQ,Gw2280,FHD,12990.0,5300.0,40.0,7690.0,4.3,439.0,2023-02-09,BenQ Gw2280 22 Inch (55.88 Cm) 1920 x 1080 Pix...,1
2,Acer,EK220Q,FHD,13750.0,7501.0,54.0,6249.0,4.2,2242.0,2023-02-05,Acer EK220Q 21.5 Inch (54.61 Cm) Full HD (1920...,1
3,Samsung,24-inch(60.46cm),FHD,19110.0,9711.0,50.0,9399.0,4.3,2709.0,2023-02-05,"Samsung 24-inch(60.46cm) FHD Monitor, IPS, 75 ...",1
4,ZEBRONICS,24,FHD,24999.0,16500.0,66.0,8499.0,3.7,233.0,2023-02-05,ZEBRONICS 24 Inch (60.4 Cm) Widescreen 1920 X ...,1
...,...,...,...,...,...,...,...,...,...,...,...,...
300,Dell,VG2455,FHD,18999.0,8259.0,43.0,10740.0,4.7,109.0,2023-02-09,ViewSonic VG2455 24 Inch IPS 1080p Monitor wit...,1
301,ViewSonic,Crossover,QHD,28800.0,11801.0,40.0,16999.0,4.5,202.0,2023-02-08,ViewSonic Crossover Monitor Vx3276-2K-Mhd-2 32...,1
303,BenQ,LED,HD,49990.0,15000.0,30.0,34990.0,4.6,551.0,2023-02-05,Acer LED Backlit Computer LCD Monitor with VGA...,1
305,BenQ,Q24H-10,QHD,119990.0,50000.0,41.0,69990.0,4.4,87.0,2023-02-06,Lenovo Q-Series Q24H-10 24 Inch(60.45Cm) LED 2...,1


In [63]:
df3.to_excel("Amazon Monitor Scrap master cleaned.xlsx", sheet_name="1", index=False)

In [71]:
res=df3.Resolution.unique().tolist()
g=df3.groupby("Resolution")
for i in range(len(res)):
    grp=g.get_group(res[i])
    df3["Company_Res"]=1

C:\Users\aakan\AppData\Local\Temp\ipykernel_12552\3754945290.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3["Company_Res"]=1


In [72]:
df3

,Company_Name,Model_Number,Resolution,Original_Price,Discount,Discount_Percentage,Discounted_Price,Average_Rating,Ratings_Count,Delivery_Date,Additional_Details,Company_sum,Company_Res
0,BenQ,GW2480,FHD,13990.0,4240.0,30.0,9750.0,4.4,18476.0,2023-02-05,"BenQ GW2480 24-Inch (60.5 Cm) 1080p, LED 1920 ...",1,1
1,BenQ,Gw2280,FHD,12990.0,5300.0,40.0,7690.0,4.3,439.0,2023-02-09,BenQ Gw2280 22 Inch (55.88 Cm) 1920 x 1080 Pix...,1,1
2,Acer,EK220Q,FHD,13750.0,7501.0,54.0,6249.0,4.2,2242.0,2023-02-05,Acer EK220Q 21.5 Inch (54.61 Cm) Full HD (1920...,1,1
3,Samsung,24-inch(60.46cm),FHD,19110.0,9711.0,50.0,9399.0,4.3,2709.0,2023-02-05,"Samsung 24-inch(60.46cm) FHD Monitor, IPS, 75 ...",1,1
4,ZEBRONICS,24,FHD,24999.0,16500.0,66.0,8499.0,3.7,233.0,2023-02-05,ZEBRONICS 24 Inch (60.4 Cm) Widescreen 1920 X ...,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,Dell,VG2455,FHD,18999.0,8259.0,43.0,10740.0,4.7,109.0,2023-02-09,ViewSonic VG2455 24 Inch IPS 1080p Monitor wit...,1,1
301,ViewSonic,Crossover,QHD,28800.0,11801.0,40.0,16999.0,4.5,202.0,2023-02-08,ViewSonic Crossover Monitor Vx3276-2K-Mhd-2 32...,1,1
303,BenQ,LED,HD,49990.0,15000.0,30.0,34990.0,4.6,551.0,2023-02-05,Acer LED Backlit Computer LCD Monitor with VGA...,1,1
305,BenQ,Q24H-10,QHD,119990.0,50000.0,41.0,69990.0,4.4,87.0,2023-02-06,Lenovo Q-Series Q24H-10 24 Inch(60.45Cm) LED 2...,1,1


In [73]:
df3.to_excel("Amazon Monitor Scrap master cleaned.xlsx", sheet_name="1", index=False)

In [3]:
df=pd.read_excel("Amazon Monitor Scrap master cleaned.xlsx", sheet_name="1")
df

,Company_Name,Model_Number,Resolution,Original_Price,Discount,Discount_Percentage,Discounted_Price,Average_Rating,Ratings_Count,Delivery_Date,Additional_Details,Company_sum,Company_Res
0,BenQ,GW2480,FHD,13990.0,4240.0,30,9750,4.4,18476,2023-02-05,"BenQ GW2480 24-Inch (60.5 Cm) 1080p, LED 1920 ...",1,1
1,BenQ,Gw2280,FHD,12990.0,5300.0,40,7690,4.3,439,2023-02-09,BenQ Gw2280 22 Inch (55.88 Cm) 1920 x 1080 Pix...,1,1
2,Acer,EK220Q,FHD,13750.0,7501.0,54,6249,4.2,2242,2023-02-05,Acer EK220Q 21.5 Inch (54.61 Cm) Full HD (1920...,1,1
3,Samsung,24-inch(60.46cm),FHD,19110.0,9711.0,50,9399,4.3,2709,2023-02-05,"Samsung 24-inch(60.46cm) FHD Monitor, IPS, 75 ...",1,1
4,ZEBRONICS,24,FHD,24999.0,16500.0,66,8499,3.7,233,2023-02-05,ZEBRONICS 24 Inch (60.4 Cm) Widescreen 1920 X ...,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,Dell,VG2455,FHD,18999.0,8259.0,43,10740,4.7,109,2023-02-09,ViewSonic VG2455 24 Inch IPS 1080p Monitor wit...,1,1
221,ViewSonic,Crossover,QHD,28800.0,11801.0,40,16999,4.5,202,2023-02-08,ViewSonic Crossover Monitor Vx3276-2K-Mhd-2 32...,1,1
222,BenQ,LED,HD,49990.0,15000.0,30,34990,4.6,551,2023-02-05,Acer LED Backlit Computer LCD Monitor with VGA...,1,1
223,BenQ,Q24H-10,QHD,119990.0,50000.0,41,69990,4.4,87,2023-02-06,Lenovo Q-Series Q24H-10 24 Inch(60.45Cm) LED 2...,1,1


In [4]:
a=df.Company_Name.unique().tolist()

In [18]:
b=df.Company_Name.unique.count.tolist()

AttributeError: 'function' object has no attribute 'count'

In [7]:
g=df.groupby("Company_Name")
to_drop=[]
c=[]
for i in range(len(a)):
    b=g.get_group(a[i]).Company_Name.count()
    c.append(b)
    

In [10]:
for i in range(len(c)):
    if c[i]<3:
        to_drop.append(a[i])

In [11]:
to_drop

['Consistent',
 'Elista',
 'AmazonBasics',
 'Amazon',
 'ZEBSTER',
 'Lapcare',
 'Lumi',
 'ARZOPA',
 'Zebster',
 'Enter-Go',
 'GIGABYTE',
 'Geonix',
 'Zebronics',
 'Mercury',
 'JIN',
 'Miracle',
 'Frontech',
 'Viewsonic',
 'Riitek']

In [16]:
for i in range(len(to_drop)):
    df=df.drop(df[df["Company_Name"]==to_drop[i]].index, axis=0)

In [17]:
df2

,Company_Name,Model_Number,Resolution,Original_Price,Discount,Discount_Percentage,Discounted_Price,Average_Rating,Ratings_Count,Delivery_Date,Additional_Details,Company_sum,Company_Res
0,BenQ,GW2480,FHD,13990.0,4240.0,30,9750,4.4,18476,2023-02-05,"BenQ GW2480 24-Inch (60.5 Cm) 1080p, LED 1920 ...",1,1
1,BenQ,Gw2280,FHD,12990.0,5300.0,40,7690,4.3,439,2023-02-09,BenQ Gw2280 22 Inch (55.88 Cm) 1920 x 1080 Pix...,1,1
2,Acer,EK220Q,FHD,13750.0,7501.0,54,6249,4.2,2242,2023-02-05,Acer EK220Q 21.5 Inch (54.61 Cm) Full HD (1920...,1,1
3,Samsung,24-inch(60.46cm),FHD,19110.0,9711.0,50,9399,4.3,2709,2023-02-05,"Samsung 24-inch(60.46cm) FHD Monitor, IPS, 75 ...",1,1
4,ZEBRONICS,24,FHD,24999.0,16500.0,66,8499,3.7,233,2023-02-05,ZEBRONICS 24 Inch (60.4 Cm) Widescreen 1920 X ...,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,Dell,VG2455,FHD,18999.0,8259.0,43,10740,4.7,109,2023-02-09,ViewSonic VG2455 24 Inch IPS 1080p Monitor wit...,1,1
221,ViewSonic,Crossover,QHD,28800.0,11801.0,40,16999,4.5,202,2023-02-08,ViewSonic Crossover Monitor Vx3276-2K-Mhd-2 32...,1,1
222,BenQ,LED,HD,49990.0,15000.0,30,34990,4.6,551,2023-02-05,Acer LED Backlit Computer LCD Monitor with VGA...,1,1
223,BenQ,Q24H-10,QHD,119990.0,50000.0,41,69990,4.4,87,2023-02-06,Lenovo Q-Series Q24H-10 24 Inch(60.45Cm) LED 2...,1,1
